In [2]:
%matplotlib qt

from roipoly import RoiPoly
from matplotlib import pyplot as plt
import cv2
from skimage import data, util
from skimage.measure import label, regionprops
import numpy as np
from PIL import Image
import math
from matplotlib import patches as mpatches

# 1 Labeling

## 1.1 Hand Labeling Appropriate Regions

In [2]:
image = cv2.imread('ECE5242Proj1-train/2_3.1.png', 0)

plt.imshow(image, cmap = "gray")

In [3]:
my_roi = RoiPoly(color='r')

In [4]:
plt.imshow(image, cmap='gray')
my_roi.display_roi()

In [5]:
mask = my_roi.get_mask(image)
plt.imshow(mask)

In [6]:
RGB_image = cv2.imread('ECE5242Proj1-train/2_3.1.png', 1)
RGB_image = cv2.cvtColor(RGB_image, cv2.COLOR_BGR2HSV)
red_image = mask.reshape(900,1200,1) * RGB_image
red_image = red_image.reshape(900 * 1200,3)
print(red_image.shape)

(1080000, 3)


In [7]:
red_image = red_image[red_image.sum(axis=(1)) != 0]/255
print(red_image.shape)

(85240, 3)


In [22]:
# plot in 3D visualization just to see points
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

reds = redBarrelRGBs[:,0]
greens = redBarrelRGBs[:,1]
blues = redBarrelRGBs[:,2]
ax.scatter(reds, greens, blues, c='red', alpha=0.01)
ax.view_init(elev=55.0, azim=70.0)

ax.set_xlabel('R')
ax.set_ylabel('G')
ax.set_zlabel('B')

plt.show()

In [21]:
fig2 = plt.figure()
ax = fig2.add_subplot(111, projection = '3d')

reds = nonRedBarrelRGBs[:,0]
greens = nonRedBarrelRGBs[:,1]
blues = nonRedBarrelRGBs[:,2]
ax.scatter(reds, greens, blues, c='gray', alpha=0.01)
ax.view_init(elev=55.0, azim=70.0)

ax.set_xlabel('R')
ax.set_ylabel('G')
ax.set_zlabel('B')

plt.show()

## 1.2 Using HSV points from Mask Generator

In [3]:
HSVPoints = np.load('hsvPoints.npy')

In [4]:
testImagesRGB = np.zeros((11,900,1200,3))

testImagesRGB[0] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/2.8.png', 1) , cv2.COLOR_BGR2RGB )
testImagesRGB[1] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/2.14.png', 1) , cv2.COLOR_BGR2RGB )
testImagesRGB[2] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/3.11.png', 1) , cv2.COLOR_BGR2RGB )
testImagesRGB[3] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/4.10.png', 1) , cv2.COLOR_BGR2RGB )
testImagesRGB[4] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/5.11.png', 1) , cv2.COLOR_BGR2RGB )
testImagesRGB[5] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/6.8.png', 1) , cv2.COLOR_BGR2RGB )
testImagesRGB[6] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/7.6.png', 1) , cv2.COLOR_BGR2RGB )
testImagesRGB[7] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/8.5.png', 1) , cv2.COLOR_BGR2RGB )
testImagesRGB[8] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/9.3.png', 1) , cv2.COLOR_BGR2RGB )
testImagesRGB[9] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/10.1.png', 1) , cv2.COLOR_BGR2RGB )
testImagesRGB[10] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/14.png', 1) , cv2.COLOR_BGR2RGB )

testImagesHSV = np.zeros((11,900,1200,3))

testImagesHSV[0] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/2.8.png', 1) , cv2.COLOR_BGR2HSV )
testImagesHSV[1] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/2.14.png', 1) , cv2.COLOR_BGR2HSV )
testImagesHSV[2] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/3.11.png', 1) , cv2.COLOR_BGR2HSV )
testImagesHSV[3] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/4.10.png', 1) , cv2.COLOR_BGR2HSV )
testImagesHSV[4] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/5.11.png', 1) , cv2.COLOR_BGR2HSV )
testImagesHSV[5] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/6.8.png', 1) , cv2.COLOR_BGR2HSV )
testImagesHSV[6] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/7.6.png', 1) , cv2.COLOR_BGR2HSV )
testImagesHSV[7] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/8.5.png', 1) , cv2.COLOR_BGR2HSV )
testImagesHSV[8] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/9.3.png', 1) , cv2.COLOR_BGR2HSV )
testImagesHSV[9] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/10.1.png', 1) , cv2.COLOR_BGR2HSV )
testImagesHSV[10] = cv2.cvtColor( cv2.imread('ECE5242Proj1-train/14.png', 1) , cv2.COLOR_BGR2HSV )

# 2 Learning Algorithms

## Simple Gaussian

In [5]:
def muCalculator(pointsArray):
    mu = np.mean(pointsArray, axis=0)
    return mu

In [6]:
def sigmaCovCalculator(pointsArray):
    mu = muCalculator(pointsArray)
    sigma = (1/pointsArray.shape[0]) * np.dot((pointsArray - mu).T , (pointsArray - mu))
    return sigma

In [7]:
def gaussian(pointsArray, mu, sigma):
    gaussian = np.zeros((pointsArray.shape[0],1))
    for pointIndex, point in enumerate(pointsArray):
        difference = (point-mu).T
        multiplier = 1/ ( math.pow((2*math.pi),(3/2)) * (math.pow( np.linalg.det(sigma) , (1/2) ) ) )
        exponent = math.exp(-0.5 * np.dot(np.dot( difference.T, np.linalg.inv(sigma)), difference ))
        output = multiplier * exponent
        gaussian[pointIndex] = output
    return gaussian

In [ ]:
## show how effective it is with simple gaussian

simpleGaussianOuptut = gaussian(red_image, muCalculator(red_image), sigmaCovCalculator(red_image))

## Gaussian Mixed Model

In [ ]:
## do we want to simplify covariance by doing a simpler gaussian? Restrict the sigmas... axis of line gaussian, diagonal covariance matrices, spherical covariance matricies
## can also simplify model by doing 1/k for each weight

In [8]:
def clusterInitialization(pointsArray, k):
    # clusters = np.asarray([]) #should have mean (clusters[k][0]) and covariance (clusters[k][1]) attached
    clusters = np.asarray([])
    mean = np.mean(pointsArray, axis=0)
    for cluster in range(k):
        mu = mean + np.random.normal(0,0.01,3)
        sigma = np.identity(3)
        clusters = np.append(clusters, {"mu": mu, "sigma": sigma})
    return clusters

In [9]:
def expectation(pointsArray, clusters):
    den = np.zeros((pointsArray.shape[0],1))

    for cluster in clusters:
        z = gaussian(pointsArray, cluster["mu"], cluster["sigma"])
        for i in range(pointsArray.shape[0]):
            den[i] += z[i]
        cluster["z"] = z
        cluster["den"] = den
    
    for clusterIndex, cluster in enumerate(clusters):
        cluster["z"] /= den

In [10]:
def maximization(pointsArray, clusters):
    for cluster in clusters:
        z = np.sum(cluster["z"], axis=0)
        mu = (1/z) * np.sum(cluster["z"] * pointsArray, axis = 0)

        cov = np.zeros((pointsArray.shape[1],pointsArray.shape[1]))
        for i in range(pointsArray.shape[0]):
            difference = (pointsArray[i] - mu).reshape(-1,1)
            cov_i = cluster["z"][i] * np.dot(difference, difference.T)
            cov = cov + cov_i
        sigma = (1/z) * cov
        
        cluster["mu"] = mu
        cluster["sigma"] = sigma

In [11]:
def logLikelihood(clusters):
    likelihoods = np.log(np.array([cluster["den"] for cluster in clusters]))
    likelihoodsSum = np.sum(likelihoods)
    return likelihoodsSum, likelihoods

In [12]:
def train_GMM(redPointsArray, k, emIterations):
    clusters = clusterInitialization(redPointsArray, k)
    likelihoodsArray = np.zeros((emIterations, ))
    history = []

    for i in range(emIterations):
        clusters_record = []
        for cluster in clusters:
            clusters_record.append(
                {"mu": cluster["mu"].copy(),
                "sigma": cluster["sigma"].copy()})
        history.append(clusters_record)

        expectation(redPointsArray, clusters)
        maximization(redPointsArray, clusters)

        likelihoodsSum, likelihoods = logLikelihood(clusters)
        likelihoodsArray[i] = likelihoodsSum

        print("emIteration: ", i+1, "likelihoodsSum: ", likelihoodsSum)

    return clusters, likelihoodsArray, history

In [ ]:
# testing for number of GMMs

In [13]:
clusters, likelihoodsArray, history = train_GMM(HSVPoints, 3, 7)

emIteration:  1 likelihoodsSum:  -2889351.461709732
emIteration:  2 likelihoodsSum:  3357455.0595450783
emIteration:  3 likelihoodsSum:  3357474.730666676
emIteration:  4 likelihoodsSum:  3358002.795151293
emIteration:  5 likelihoodsSum:  3374826.927578327
emIteration:  6 likelihoodsSum:  3515039.151140819
emIteration:  7 likelihoodsSum:  3739317.736755443


In [15]:
history

[[{'mu': array([0.58780253, 0.7447029 , 0.67631085]),
   'sigma': array([[1., 0., 0.],
          [0., 1., 0.],
          [0., 0., 1.]])},
  {'mu': array([0.58665018, 0.74328876, 0.6597357 ]),
   'sigma': array([[1., 0., 0.],
          [0., 1., 0.],
          [0., 0., 1.]])},
  {'mu': array([0.58187183, 0.74755552, 0.67379614]),
   'sigma': array([[1., 0., 0.],
          [0., 1., 0.],
          [0., 0., 1.]])}],
 [{'mu': array([0.57720382, 0.74678462, 0.67256418]),
   'sigma': array([[ 0.06791526,  0.0044519 , -0.00461021],
          [ 0.0044519 ,  0.00976644, -0.00528649],
          [-0.00461021, -0.00528649,  0.05392756]])},
  {'mu': array([0.57719562, 0.74685315, 0.67168248]),
   'sigma': array([[ 0.06791674,  0.00444682, -0.00460296],
          [ 0.00444682,  0.00976073, -0.00526804],
          [-0.00460296, -0.00526804,  0.05400186]])},
  {'mu': array([0.57682484, 0.74679935, 0.67244084]),
   'sigma': array([[ 0.06808727,  0.00445631, -0.00462004],
          [ 0.00445631,  0.009767

In [16]:
clusters, likelihoodsArray, history = train_GMM(HSVPoints, 2, 7)

emIteration:  1 likelihoodsSum:  -2379188.1983924625
emIteration:  2 likelihoodsSum:  1785309.829404751
emIteration:  3 likelihoodsSum:  1785540.6678579182
emIteration:  4 likelihoodsSum:  1793528.2182930126
emIteration:  5 likelihoodsSum:  1859315.0222701218
emIteration:  6 likelihoodsSum:  1973009.5122277504
emIteration:  7 likelihoodsSum:  2072798.3931222847


In [18]:
clusters, likelihoodsArray, history = train_GMM(HSVPoints, 2, 30)

emIteration:  1 likelihoodsSum:  -2379224.160074328
emIteration:  2 likelihoodsSum:  1785315.758318486
emIteration:  3 likelihoodsSum:  1785505.122126888
emIteration:  4 likelihoodsSum:  1791463.3136595474
emIteration:  5 likelihoodsSum:  1853445.2391817598
emIteration:  6 likelihoodsSum:  1990858.4000144969
emIteration:  7 likelihoodsSum:  2243673.148530864
emIteration:  8 likelihoodsSum:  3647673.8463638956
emIteration:  9 likelihoodsSum:  4837147.970206054
emIteration:  10 likelihoodsSum:  4893165.5696677845
emIteration:  11 likelihoodsSum:  4920095.1676156875
emIteration:  12 likelihoodsSum:  4939751.762706778
emIteration:  13 likelihoodsSum:  4958626.016163501
emIteration:  14 likelihoodsSum:  4974304.67717166
emIteration:  15 likelihoodsSum:  4985573.5024667885
emIteration:  16 likelihoodsSum:  4997323.372385021
emIteration:  17 likelihoodsSum:  5017281.354893607
emIteration:  18 likelihoodsSum:  5048812.951268961
emIteration:  19 likelihoodsSum:  5090307.017329614
emIteration:  

In [21]:
plt.figure(figsize=(10,10))
plt.title('Log Likelihood')
plt.plot(np.arange(1, 31), likelihoodsArray)
plt.xlabel("EM Iterations")
plt.ylabel("Log Likelihoods Sum")
plt.show()

In [22]:
likelihoodsArray

array([-2379224.16007433,  1785315.75831849,  1785505.12212689,
        1791463.31365955,  1853445.23918176,  1990858.4000145 ,
        2243673.14853086,  3647673.8463639 ,  4837147.97020605,
        4893165.56966778,  4920095.16761569,  4939751.76270678,
        4958626.0161635 ,  4974304.67717166,  4985573.50246679,
        4997323.37238502,  5017281.35489361,  5048812.95126896,
        5090307.01732961,  5108260.17037649,  5110204.17728829,
        5110550.93349635,  5110585.56130762,  5110591.65575578,
        5110597.76133484,  5110602.21920356,  5110603.06932463,
        5110603.10568719,  5110603.10646624,  5110603.10648072])

In [13]:
def calculateGMMProbabilities(inputImage, clusters):
    GMMOutput = np.zeros((inputImage.shape[0], 1))
    for cluster in clusters:
        simpleGaussianOutput = gaussian(inputImage, cluster["mu"], cluster["sigma"])
        # print(np.sum(simpleGaussianOutput))
        GMMOutput = GMMOutput + ((1/clusters.shape[0]) * simpleGaussianOutput)
    return GMMOutput

# Testing Image

## Applying Test Image

In [179]:
# run with all the test images to get ratio for distance calc
ratios = np.array([])

In [377]:
# make sure the input image is reshaped and normalized
input_image = testImagesHSV[11].reshape(900*1200, 3)
input_image = input_image[input_image.sum(axis=(1)) != 0]/255

# do the same to RGB version for plotting later
input_image_RGB = testImagesRGB[11].reshape(900*1200, 3)
input_image_RGB = input_image_RGB[input_image_RGB.sum(axis=(1)) != 0]/255

GMMOutput = calculateGMMProbabilities(input_image, clusters)

plt.imshow(GMMOutput.reshape(900,1200))

IndexError: index 11 is out of bounds for axis 0 with size 11

In [369]:
# image erosion and dilation
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
GMMOutput = np.uint8(GMMOutput)

# GMMOutput = cv2.erode(GMMOutput, kernel, iterations = 3)
# # GMMOutput = cv2.morphologyEx(GMMOutput, cv2.MORPH_CLOSE, kernel)
# GMMOutput = cv2.dilate(GMMOutput, kernel, iterations = 3)

# plt.imshow(GMMOutput.reshape(900,1200))

In [370]:
GMMOutput = cv2.erode(GMMOutput, kernel, iterations = 1)
plt.imshow(GMMOutput.reshape(900,1200))

In [371]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
GMMOutput = cv2.dilate(GMMOutput, kernel, iterations = 1)

plt.imshow(GMMOutput.reshape(900,1200))

In [372]:
# replace image with binary (1,0) for white and black
GMMOutput = np.where( GMMOutput > 0.99, 1, 0)
plt.imshow(GMMOutput.reshape(900,1200))

In [373]:
# plot labels on gray
label_img = label(GMMOutput.reshape(900,1200), connectivity=2)
plt.imshow(label_img, cmap='gray')

In [390]:
# PUTS A BOUNDING BOX AROUND BARRELL
# distance = 14

fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(input_image_RGB.reshape(900, 1200, 3))
# regionsArray = []
for region in regionprops(label_img):
    if region.area >= 1200:
        bboxAspect = (region.bbox[2]-region.bbox[0])/(region.bbox[3]-region.bbox[1])
        if (bboxAspect > 1.0325 and bboxAspect < 1.9528):
            # regionsArray.append(region)
            print(region.area)
            print(region.bbox)
            minr, minc, maxr, maxc = region.bbox
            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                    fill=False, edgecolor='green', linewidth=2)
            ax.add_patch(rect)
            ratio = 493.0884217183345
            barrelDistance = str( round(ratio/ math.sqrt(region.area) , 2)) + 'm'

            print(region.centroid)
            plt.annotate(barrelDistance, (region.centroid[1], region.centroid[0]), color='white')

            # # calculating distance
            # # True Distance * sqrt(region.area) gives a ratio
            # ratio = math.sqrt(region.area) * distance
            # print(ratio)
            # ratios = np.append(ratios, ratio)

            # now with distance ratio calculated (493.0884217183345), appending distance

plt.show()

1134
(195, 600, 239, 627)
(216.69223985890653, 613.1640211640212)


In [376]:
ratios

array([524.00763353, 551.43721311, 488.26222463, 498.14656478,
       519.54595562, 502.10058753, 532.22551611, 491.63401021,
       453.10043037, 370.42408129])

In [344]:
ratios = np.delete(ratios, 7)

In [378]:
ratiosMean = np.mean(ratios)

In [379]:
ratiosMean

493.0884217183345

## Future Optimizations

In [ ]:
# TO DO: 
# optimize this with row dialation (region props)
# cv2 erode & dialate?

In [ ]:
clusters, likelihoodsArray, history = train_GMM(HSVPoints, 2, 9)

In [393]:
np.save('clusters.npy', clusters)

In [394]:
clusters.shape

(2,)